In [ ]:
# https://stepik.org/lesson/628341/step/1?unit=624221

In [1]:
# Because we have not released Lock, 2nd process could not start.
# We can release such lock from anywhere, for example from the main process in interactive mode
import multiprocessing
import time

lock = multiprocessing.Lock()

def get_value(l):
    l.acquire()
    pr_name = multiprocessing.current_process().name
    print(f'Process [{pr_name}] started')

multiprocessing.Process(target=get_value, args=(lock,)).start()
multiprocessing.Process(target=get_value, args=(lock,)).start()
time.sleep(1)  # this is needed for Jupiter (it stops output after main process ends)

Process [Process-1] started


In [2]:
# RLock - similar to previous example, but we can release lock only from process where it has been acquired.
import multiprocessing
import time

lock = multiprocessing.RLock()

def get_value(l):
    l.acquire()
    pr_name = multiprocessing.current_process().name
    print(f'Process [{pr_name}] started')

multiprocessing.Process(target=get_value, args=(lock,)).start()
multiprocessing.Process(target=get_value, args=(lock,)).start()
time.sleep(1)  # this is needed for Jupiter (it stops output after main process ends)

Process [Process-3] started


In [5]:
# The multiprocessing.Array class allows to create shared arrays between multiple processes.
# A shared array is an array that can be accessed and modified by different processes concurrently.
# This can be beneficial when you need to share large data structures, such as arrays,
# between processes without incurring the overhead of inter-process communication.

# If we don't use locker here resulted array still be correctly filled (because the Array is concurrent safe),
# but time.sleep will count time simultaneously for all processes.

import multiprocessing
import random
import time

def add_value(locker, array, index):
    with locker:
        num = random.randint(0, 5)
        vtime = time.ctime()
        array[index] = num
        print(f'array[{index}] = {num}, time = {vtime}')
        time.sleep(num)



arr = multiprocessing.Array('i', range(10))
print(arr)
print(list(arr))
lock = multiprocessing.Lock()
processes = []

for i in range(10):
    pr = multiprocessing.Process(target=add_value, args=(lock, arr, i,))
    processes.append(pr)
    pr.start()

for pr in processes:
    pr.join()

print(list(arr))

<SynchronizedArray wrapper for <multiprocessing.sharedctypes.c_int_Array_10 object at 0x7f96542f0c20>>
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
array[0] = 3, time = Wed May 31 00:37:28 2023
array[1] = 0, time = Wed May 31 00:37:28 2023
array[2] = 5, time = Wed May 31 00:37:28 2023
array[3] = 1, time = Wed May 31 00:37:28 2023
array[4] = 3, time = Wed May 31 00:37:28 2023
array[5] = 3, time = Wed May 31 00:37:28 2023
array[6] = 3, time = Wed May 31 00:37:28 2023
array[7] = 1, time = Wed May 31 00:37:28 2023
array[8] = 5, time = Wed May 31 00:37:28 2023
array[9] = 5, time = Wed May 31 00:37:28 2023
[3, 0, 5, 1, 3, 3, 3, 1, 5, 5]


In [6]:
# The multiprocessing.Queue is designed to be used in a multi-process environment,
# where one process can put items into the queue while another process can retrieve them.

import multiprocessing

def get_text(q):
    q.put('test')

queue = multiprocessing.Queue()
pr = multiprocessing.Process(target=get_text, args=(queue,))

pr.start()
print(queue.get())
pr.join()


test


In [10]:
import multiprocessing
import random

def get_text(q):
    val = random.randint(0, 10)
    q.put(str(val))

queue = multiprocessing.Queue()
processes = []

for _ in range(10):
    pr = multiprocessing.Process(target=get_text, args=(queue,))
    processes.append(pr)
    pr.start()

for pr in processes:
    pr.join()

queue.put(None)  # sentinel value for iter()

for elem in iter(queue.get, None):
    print(elem)

5
0
8
5
0
2
8
8
8
7
